# Preliminary Experiment
To begin exploring the relationships in our data we decided to apply k-means clustering analysis. 

In [ ]:
# Imports
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sklearn as skl
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn import set_config
set_config(transform_output="pandas")
%matplotlib inline

SEED = 42
np.random.seed(SEED)

Loading the cleaned data

Setting up a sklearn pipeline

Conducting clustering 